In [69]:
import os
import pandas as pd
directory_path = 'comments/'
data_frames={}
for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
        data_frames[filename]=(pd.read_csv(os.path.join(directory_path, filename)))


In [70]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest",truncation=True,max_length=512)
pipe2 = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis",truncation=True,max_length=128)
pipe3 = pipeline("text-classification", model="Seethal/sentiment_analysis_generic_dataset",truncation=True,max_length=512)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
def convert_model2(x):
    if x=='POS':
        return 'positive'
    elif x=='NEG':
        return 'negative'
    else:
        return 'neutral'
def convert_model3(x):
    if x=='LABEL_0':
        return 'negative'
    elif x=='LABEL_1':
        return 'neutral'
    else:
        return 'positive'

In [72]:
from tqdm import tqdm
for i in tqdm(data_frames.keys()):
    predictions_model1=pipe(list(data_frames[i]["Comment Text"]))
    predictions_model2=pipe2(list(data_frames[i]["Comment Text"]))
    predictions_model3=pipe3(list(data_frames[i]["Comment Text"]))
    model1=[]
    model2=[]
    model3=[]
    for j in range(len(predictions_model1)):
        model1.append(predictions_model1[j]['label'])
        model2.append(convert_model2(predictions_model2[j]['label']))
        model3.append(convert_model3(predictions_model3[j]['label']))
    data_frames[i]['model1']=model1
    data_frames[i]['model2']=model2
    data_frames[i]['model3']=model3

100%|██████████| 100/100 [33:06<00:00, 19.86s/it]


In [75]:
for i in data_frames.keys():
    majority_labels=[]
    for j in range(len(data_frames[i])):
        labels=[data_frames[i]['model1'][j],data_frames[i]['model2'][j],data_frames[i]['model3'][j]]
        majority_labels.append(max(set(labels), key = labels.count))
    data_frames[i]['majority_labels']=majority_labels

In [76]:
for i in data_frames.keys():
    data_frames[i].to_csv('majority/'+i,index=False)